# Matrix factorization
## System of equations solving
---

Let´s consider the following sistem of $n$ linear equations with $n$ variables:

$$A\mathbf{x} = \begin{pmatrix}
a_{11} & a_{12} & \cdots & a_{1n} \\
a_{21} & a_{22} & \cdots & a_{2n} \\
\vdots & \vdots & \ddots & \vdots \\
a_{n1} & a_{n2} & \cdots & a_{nn}
\end{pmatrix} \begin{pmatrix}
x_1 \\
x_2 \\
\vdots \\
x_n
\end{pmatrix}= \begin{pmatrix}
b_1 \\
b_2 \\
\vdots \\
b_n
\end{pmatrix} =\mathbf{b}$$

where $A\in\mathbb{R}^{n\times n}$ and $\mathbf{x},\mathbf{b}\in \mathbb{R}^n$. For small matrices, there are convenient methods that can be used directly to find the values ​​of the variables. However, when the size ($n$) increases significantly, it becomes difficult to determine whether a system has a solution and, if so, to find it. For this reason, several mathematicians found ways to factor the system in order to simplify it. In this notebook, I will develop some of these methods, along with their computational implementation programmed from scratch, since within data science we can find various situations (when making predictions, forecasts, and regressions) in which it is necessary to solve one of these problems quickly and with the least possible error.

### Libraries

In [11]:
import numpy as np
from scipy.linalg import lu

### LU Factorization

A first approach involves finding two matrices $L, U \in\mathbb{R}^{n\times n}$ such that $A = LU$, $L$ is a lower triangular matrix and $U$, on the contrary, is an upper triangular matrix. Thus, our system becomes $L\mathbf{y} = \mathbf{b}$, where $U\mathbf{x}=\mathbf{y}$. The main requirement for this factorization to exist is that $A$ must be strictly invertible. Its operation is approximately as follows:

In [4]:
def LU_factorization(A):
    """
    Function that finds LU factorization through Doolittle method.

    inputs:
    --------
        A - NumPy array: Matrix to be factored. It must be square and invertible (with a determinant other than 0).

    outputs:
    --------
        L - NumPy array: Lower triangular matrix of the factorization. This matrix has 1s on the diagonal.
        U - NumPy array: Upper triangular matrix of the factorization.
    """
    m, n = A.shape
    if m != n:
        raise ValueError("Matrix must be square.")
    L = np.eye(m, n)
    U = np.zeros((m, n))

    for i in range(n):
        for j in range(i, n):
            sum_ = 0.0
            for k in range(i):
                sum_ += L[i][k] * U[k][j]
            U[i][j] = A[i][j] - sum_
        for j in range(i+1, n):
            sum_ = 0.0
            for k in range(i):
                sum_ += L[j][k] * U[k][i]
            if abs(U[i][i]) < 1e-16:
                raise ValueError("Matrix must be invertible.")
            else:
                L[j][i] = (A[j][i] - sum_) / U[i][i]
    return L, U



Note that the previous solution has a drawback: If `U[i][i]` is very small (or even 0), the factorization could result in an error or return very large numbers in the inputs of the factored matrices, implying a high computational cost. Therefore, it is convenient for the diagonal elements of the original matrix to be the largest available. Thus, we can permute the rows using a matrix $P$ before performing the factorization, modifying the previous logic to $A=PLU$:

In [16]:
def PLU_factorization(A):
    """
    Function that finds PLU factorization through gaussian elimination.

    inputs:
    --------
        A - NumPy array: Matrix to be factored. It must be square and invertible (with a determinant other than 0).

    outputs:
    --------
        P - NumPy array: Permutation matrix
        L - NumPy array: Lower triangular matrix of the factorization. This matrix has 1s on the diagonal.
        U - NumPy array: Upper triangular matrix of the factorization.    
    """
    m, n = A.shape
    if m != n:
        raise ValueError("Matrix must be square.")
    U = A.copy().astype(float)
    L = np.eye(n)
    P = np.eye(n)

    for k in range(n-1):
        max_index = np.argmax(np.abs(U[k:, k])) + k
        if max_index != k:
            U[[k, max_index], k:] = U[[max_index, k], k:]
            P[[k, max_index]] = P[[max_index, k]]
            if k > 0:
                L[[k, max_index], :k] = L[[max_index, k], :k]
        if abs(U[k, k]) < 1e-15:
            raise ValueError("The matrix is ​​singular, it cannot be factored.")
        rows = slice(k+1, n)
        L[rows, k] = U[rows, k] / U[k, k]
        changes = L[rows, k].reshape(-1, 1) * U[k, k:]
        U[rows, k:] -= changes
    return P, L, U

Within the linear algebra tools of `SciPy`, we can find an implementation of this factorization. We can then compare our result with that of Python:

In [23]:
A = np.array([[1, 2], [3, 4]])
P, L, U = PLU_factorization(A)
P_s, L_s, U_s = lu(A)
print(P, "\n", L, "\n", U)
print(P_s, "\n", L_s, "\n", U_s)
print(np.linalg.norm(A - P @ L @ U))
print(np.linalg.norm(A - P.T @ L_s @ U_s))


[[0. 1.]
 [1. 0.]] 
 [[1.         0.        ]
 [0.33333333 1.        ]] 
 [[3.         4.        ]
 [0.         0.66666667]]
[[0. 1.]
 [1. 0.]] 
 [[1.         0.        ]
 [0.33333333 1.        ]] 
 [[3.         4.        ]
 [0.         0.66666667]]
0.0
0.0
